# AI → LinuxCLI via MCP

This notebook module teaches how to build a **minimal MCP-like** (Model Context Protocol) service in **C** and how to use it from a **Python** agent. It is designed for systems programming courses where students already know socket basics (FTP) and want to modernize to AI-enabled tools.

**Important:** This notebook executes shell commands and writes files. Run it in an isolated VM/container for student labs. The notebook writes two files to `/mnt/data/`:
- `server.c` — the educational MCP-like C server
- `client.py` — a Python MCP client/agent example


## Learning Goals

1. Implement a socket-based server in C that understands newline-delimited JSON requests.
2. Advertise tools (with simple schemas) and execute safe, whitelisted Linux CLI actions.
3. Build a Python client/agent that performs `initialize`, `list_tools`, and `call_tool` interactions.
4. Understand security trade-offs and how to harden the service for labs.


## What is MCP? (Concise)

- MCP (Model Context Protocol) is an application-level JSON-based protocol for AI agents to discover and call external tools.
- Messages are typically newline-delimited JSON objects over streams (TCP/WebSocket/stdio).
- Key message types: `initialize`, `list_tools`, `call_tool`, `notifications`/`progress`.

This notebook implements a minimal MCP-like subset (initialize + list_tools + call_tool) focused on mapping AI tool calls to Linux CLI commands while teaching sockets programming.

## Handshake & Tool Call Flow (ASCII diagram)

```
Client                     MCP Server
------                     ----------
   ---> initialize  -------------------->
   <--- initialize_result ---------------
   ---> list_tools --------------------->
   <--- list_tools_result --------------
   ---> call_tool(list_files) -----------
   <--- tool_result ----------------------
```

Messages are newline-delimited JSON objects, and requests include an `id` that the server echoes in the response for correlation.

## Implementation Overview

- **Server language:** C (sockets, fork, basic argument handling).
- **Client language:** Python (agent example + mocked LLM).
- **JSON parsing:** For clarity the starter server uses simple string extraction and sanitization for a few fields — students should replace these with a proper parser like `cJSON` or `jsmn` in later labs.
- **Security:** The server whitelists tools and sanitizes arguments; still run in an isolated environment. Production systems MUST use TLS + auth and robust JSON handling.


In [ ]:
# This cell writes the `server.c` educational implementation into /mnt/data/server.c
server_c = r'''/*
 * server.c
 * Minimal MCP-like TCP server for teaching.
 * - Accepts a client and forks a child to serve it.
 * - Newline-delimited JSON messages.
 * - Implements: initialize, list_tools, call_tool (list_files, get_time, delete_older_than_days)
 *
 * WARNING: Educational code. Replace naive JSON handling with cJSON/jsmn for production.
 */

#define _GNU_SOURCE
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <errno.h>
#include <arpa/inet.h>
#include <time.h>
#include <sys/types.h>
#include <sys/socket.h>
#include <sys/wait.h>
#include <ctype.h>

#define PORT 9000
#define BACKLOG 4
#define BUFSIZE 16384
#define MAX_ARG 512

int send_json(int fd, const char *json) {
    size_t len = strlen(json);
    if (write(fd, json, len) != (ssize_t)len) return -1;
    if (write(fd, "\n", 1) != 1) return -1;
    return 0;
}

void sanitize_path(const char *in, char *out, size_t outsz) {
    size_t i=0;
    for (; *in && i+1 < outsz; ++in) {
        if (isalnum((unsigned char)*in) || *in=='/' || *in=='.' || *in=='_' || *in=='-' || *in==' ') {
            out[i++] = *in;
        }
    }
    out[i]=0;
}

int extract_field(const char *buf, const char *field, char *out, size_t outsz) {
    const char *p = strstr(buf, field);
    if (!p) return 0;
    p = strchr(p, ':');
    if (!p) return 0;
    p++;
    while (*p && (*p==' ' || *p=='"')) ++p;
    size_t i=0;
    while (*p && *p != '"' && *p != ',' && *p != '}' && *p != '\n' && i+1 < outsz) {
        out[i++] = *p++;
    }
    out[i]=0;
    while(i>0 && (out[i-1]==' ')) out[--i]=0;
    return 1;
}

void tool_list_files(int client_fd, const char *params) {
    char rawpath[MAX_ARG] = ".";
    if (extract_field(params, "\"path\"", rawpath, sizeof(rawpath))) {}
    char path[MAX_ARG];
    sanitize_path(rawpath, path, sizeof(path));
    if (strlen(path)==0) strcpy(path, ".");

    char cmd[2048];
    snprintf(cmd, sizeof(cmd), "ls -la --color=never %s 2>&1", path);
    FILE *fp = popen(cmd, "r");
    if (!fp) {
        send_json(client_fd, "{\"id\":null,\"type\":\"error\",\"error\":\"ls failed\"}");
        return;
    }
    send_json(client_fd, "{\"id\":null,\"type\":\"notification\",\"event\":\"tool_progress\",\"message\":\"listing files\"}");
    char output[BUFSIZE];
    output[0]=0;
    char line[1024];
    while (fgets(line, sizeof(line), fp)) {
        for (char *q=line; *q; ++q) {
            if (*q=='\"' || *q=='\\') *q=' ';
        }
        strncat(output, line, sizeof(output)-strlen(output)-1);
    }
    pclose(fp);
    char out[BUFSIZE*2];
    snprintf(out, sizeof(out),
        "{\"id\":1,\"type\":\"response\",\"result\":{\"tool\":\"list_files\",\"output\":\"%s\"}}",
        output);
    send_json(client_fd, out);
}

void tool_get_time(int client_fd) {
    time_t t = time(NULL);
    struct tm tm = *localtime(&t);
    char buf[200];
    snprintf(buf, sizeof(buf), "%04d-%02d-%02d %02d:%02d:%02d",
             tm.tm_year+1900, tm.tm_mon+1, tm.tm_mday,
             tm.tm_hour, tm.tm_min, tm.tm_sec);
    char out[512];
    snprintf(out, sizeof(out),
        "{\"id\":1,\"type\":\"response\",\"result\":{\"tool\":\"get_time\",\"time\":\"%s\"}}",
        buf);
    send_json(client_fd, out);
}

void tool_delete_older(int client_fd, const char *params) {
    char rawpath[MAX_ARG] = ".";
    char days_s[32] = "0";
    if (extract_field(params, "\"path\"", rawpath, sizeof(rawpath))) {}
    extract_field(params, "\"days\"", days_s, sizeof(days_s));
    char path[MAX_ARG];
    sanitize_path(rawpath, path, sizeof(path));
    int days = atoi(days_s);
    if (days <= 0) {
        send_json(client_fd, "{\"id\":1,\"type\":\"response\",\"result\":{\"error\":\"invalid days value\"}}");
        return;
    }
    char cmd[2048];
    snprintf(cmd, sizeof(cmd), "find %s -maxdepth 1 -type f -mtime +%d -print -delete 2>&1", path, days);
    FILE *fp = popen(cmd, "r");
    if (!fp) {
        send_json(client_fd, "{\"id\":null,\"type\":\"error\",\"error\":\"find failed\"}");
        return;
    }
    char output[BUFSIZE];
    output[0]=0;
    char line[1024];
    while (fgets(line, sizeof(line), fp)) {
        for (char *q=line; *q; ++q) if (*q=='\"' || *q=='\\') *q=' ';
        strncat(output, line, sizeof(output)-strlen(output)-1);
    }
    pclose(fp);
    char out[BUFSIZE*2];
    snprintf(out, sizeof(out),
        "{\"id\":1,\"type\":\"response\",\"result\":{\"tool\":\"delete_older_than_days\",\"output\":\"%s\"}}",
        output);
    send_json(client_fd, out);
}

int main() {
    int sockfd, newfd;
    struct sockaddr_in serv_addr, cli_addr;
    socklen_t sin_size;
    char buf[BUFSIZE];
    int yes = 1;

    if ((sockfd = socket(AF_INET, SOCK_STREAM, 0)) == -1) {
        perror("socket");
        exit(1);
    }

    if (setsockopt(sockfd, SOL_SOCKET, SO_REUSEADDR, &yes, sizeof(int)) == -1) {
        perror("setsockopt");
        exit(1);
    }

    serv_addr.sin_family = AF_INET;
    serv_addr.sin_addr.s_addr = INADDR_ANY;
    serv_addr.sin_port = htons(PORT);
    memset(&(serv_addr.sin_zero), '\0', 8);

    if (bind(sockfd, (struct sockaddr *)&serv_addr, sizeof(struct sockaddr)) == -1) {
        perror("bind");
        exit(1);
    }

    if (listen(sockfd, BACKLOG) == -1) {
        perror("listen");
        exit(1);
    }

    printf("MCP-like server listening on port %d\n", PORT);

    while (1) {
        sin_size = sizeof(struct sockaddr_in);
        if ((newfd = accept(sockfd, (struct sockaddr *)&cli_addr, &sin_size)) == -1) {
            perror("accept");
            continue;
        }
        printf("Accepted connection from %s\n", inet_ntoa(cli_addr.sin_addr));

        pid_t pid = fork();
        if (pid == 0) {
            close(sockfd);
            ssize_t numbytes;
            while ((numbytes = read(newfd, buf, BUFSIZE-1)) > 0) {
                buf[numbytes] = 0;
                printf("Received: %s\n", buf);

                if (strstr(buf, "\"method\":\"initialize\"") || strstr(buf, "\"method\": \"initialize\"")) {
                    send_json(newfd, "{\"id\":1,\"type\":\"response\",\"result\":{\"server\":\"LinuxCLI MCP Server\",\"version\":\"1.0\"}}");
                } else if (strstr(buf, "\"method\":\"list_tools\"") || strstr(buf, "\"method\": \"list_tools\"")) {
                    send_json(newfd, "{\"id\":1,\"type\":\"response\",\"result\":{\"tools\":[{\"name\":\"list_files\",\"desc\":\"List files in a directory\",\"schema\":{\"path\":\"string\"}},{\"name\":\"get_time\",\"desc\":\"Get server time\",\"schema\":{}},{\"name\":\"delete_older_than_days\",\"desc\":\"Delete files older than N days\",\"schema\":{\"path\":\"string\",\"days\":\"integer\"}}]}}");
                } else if (strstr(buf, "\"method\":\"call_tool\"") || strstr(buf, "\"method\": \"call_tool\"")) {
                    if (strstr(buf, "\"list_files\"")) {
                        tool_list_files(newfd, buf);
                    } else if (strstr(buf, "\"get_time\"")) {
                        tool_get_time(newfd);
                    } else if (strstr(buf, "\"delete_older_than_days\"")) {
                        tool_delete_older(newfd, buf);
                    } else {
                        send_json(newfd, "{\"id\":1,\"type\":\"response\",\"result\":{\"error\":\"unknown tool\"}}");
                    }
                } else {
                    send_json(newfd, "{\"id\":null,\"type\":\"error\",\"error\":\"unknown method\"}");
                }
            }
            close(newfd);
            printf("Client disconnected\n");
            exit(0);
        } else if (pid > 0) {
            close(newfd);
            while (waitpid(-1, NULL, WNOHANG) > 0) {}
        } else {
            perror("fork");
            close(newfd);
        }
    }
    return 0;
}
'''

open('/mnt/data/server.c', 'w').write(server_c)
print('Wrote /mnt/data/server.c')


### Compile the server (run in a shell cell)

```bash
cd /mnt/data
gcc server.c -o mcp_server -std=c11
```

If `gcc` is not available in your environment, run this notebook in a container or VM with build tools installed.

In [ ]:
# This cell writes the client.py file into /mnt/data/client.py
client_py = r'''# client.py - minimal MCP client example
import socket, json, time

HOST = "127.0.0.1"
PORT = 9000

def send_msg(s, obj):
    s.send((json.dumps(obj) + "\n").encode())

def recv_msg(s, timeout=5.0):
    s.settimeout(timeout)
    data = b""
    try:
        while True:
            chunk = s.recv(4096)
            if not chunk:
                break
            data += chunk
            if b"\n" in chunk:
                break
    except socket.timeout:
        pass
    if not data:
        return None
    try:
        return json.loads(data.decode().strip())
    except Exception as e:
        return data.decode()

if __name__ == '__main__':
    s = socket.socket()
    s.connect((HOST, PORT))
    print("Connected to server")
    send_msg(s, {"id":1, "method":"initialize", "params":{}})
    print("init ->", recv_msg(s))
    send_msg(s, {"id":2, "method":"list_tools", "params":{}})
    print("tools ->", recv_msg(s))
    # call list_files on current directory
    send_msg(s, {"id":3, "method":"call_tool", "params":{"tool":"list_files","args":{"path":"."}}})
    print("list_files ->", recv_msg(s))
    # call get_time
    send_msg(s, {"id":4, "method":"call_tool", "params":{"tool":"get_time","args":{}}})
    print("get_time ->", recv_msg(s))
    # Example: delete older than 30 days (use with caution)
    # send_msg(s, {"id":5, "method":"call_tool", "params":{"tool":"delete_older_than_days","args":{"path":".","days":30}}})
    # print("delete ->", recv_msg(s))
    s.close()
'''

open('/mnt/data/client.py','w').write(client_py)
print('Wrote /mnt/data/client.py')


### Run the client (in a separate shell while the server is running)
```
python3 /mnt/data/client.py
```
You should see JSON-style responses for `initialize`, `list_tools`, `list_files` and `get_time`.

## Mocked AI Agent Example (Python)

The following snippet demonstrates how an agent (your AI shell) could discover tools using `list_tools`, form a prompt for a model, receive a JSON tool call from the model, then forward the `call_tool` to the MCP server. The example below **mocks** the model output for offline testing (no API keys required).

In [ ]:
# Agent sketch: mocked LLM output to demonstrate flow
import socket, json

HOST = '127.0.0.1'
PORT = 9000

def send_msg(s, obj):
    s.send((json.dumps(obj) + '\n').encode())

def recv_msg(s):
    data = b''
    while True:
        chunk = s.recv(4096)
        if not chunk:
            break
        data += chunk
        if b'\n' in chunk:
            break
    if not data:
        return None
    try:
        return json.loads(data.decode().strip())
    except Exception:
        return data.decode()

def mocked_model_decision(user_request, tools):
    # A tiny heuristic mock: if the user asks for '.c' files, return list_files
    if '.c' in user_request or 'C files' in user_request or ' .c' in user_request:
        return { 'tool': 'list_files', 'args': { 'path': '.' } }
    if 'time' in user_request:
        return { 'tool': 'get_time', 'args': {} }
    return { 'tool': 'list_files', 'args': { 'path': '.' } }

def agent_flow(user_request):
    s = socket.socket()
    s.connect((HOST, PORT))
    send_msg(s, {'id':1, 'method':'list_tools', 'params':{}})
    tools = recv_msg(s)
    print('tools:', tools)
    # Mock model: decide which tool to call
    decision = mocked_model_decision(user_request, tools)
    print('model decision (mocked):', decision)
    # Forward as call_tool
    send_msg(s, {'id':10, 'method':'call_tool', 'params': {'tool': decision['tool'], 'args': decision['args']}})
    result = recv_msg(s)
    print('tool result:', result)
    s.close()

if __name__ == '__main__':
    print('Agent example: list .c files (mocked)')
    agent_flow('List all .c source files in my current directory')


## Running a real agent:

Then call it from the OpenAI CLI:
```
openai api chat.completions.create --model gpt-4o \
    --mcp-server http://localhost:5000 \
    -m "List detected objects from the camera and describe the scene."
```

## Lab Assignments & Grading Rubric

**Lab 1 — Minimal MCP Server (40%)**
- Starting point: `/mnt/data/server.c` included in this notebook.
- Requirements: implement `initialize`, `list_tools`, and `call_tool` for `list_files` and `get_time`.
- Demonstrate with `/mnt/data/client.py`.

**Lab 2 — Additional Tool (30%)**
- Implement `delete_older_than_days` safely.
- Validate arguments and add confirmation flow (e.g., a `dry_run` param or confirmation token).

**Lab 3 — Agent Integration (30%)**
- Build an agent that fetches `list_tools` and issues `call_tool` from natural language (mocked LLM allowed).
- Show end-to-end natural-language → tool selection → tool execution → result formatting.

Extra credit: add TLS and token-based auth, replace naive parsing with `cJSON/jsmn`, or sandbox tool execution further (namespaces, chroot, containers).


## Security & Operational Notes

- **Isolation:** Run the server in a VM or container for labs.
- **Sanitization:** The included server does basic sanitization. For robust safety use `execve()` with argv arrays and strict validation.
- **JSON:** Replace string tricks with a proper JSON library (`cJSON` or `jsmn`) in real assignments.
- **Transport security:** Use TLS or run over an authenticated tunnel for any real networked deployment.


## Suggested in-class demo plan (10–15 minutes)
1. Start the compiled server in a terminal: `./mcp_server`.
2. Run `/mnt/data/client.py` to show `initialize` + `list_tools` + `list_files`.
3. Run the mocked agent cell to show how the agent chooses a tool and the server executes it.
4. Short discussion: how this maps to your previous FTP assignment, and why MCP is the next layer up.


## Next steps / Extensions
- Replace the server's string parsing with `cJSON` or `jsmn` and demonstrate more robust schema validation.
- Add WebSocket transport and TLS.
- Add authentication tokens and connect the server to a real LLM in a controlled lab environment.
- Extend tools to expose sensors or hardware (e.g., GPIO) for robotics labs.
